<a href="https://colab.research.google.com/github/MarcelodeFreitas/udemy_deep_learning_pytorch_python/blob/main/Projeto_3_Classifica%C3%A7%C3%A3o_bin%C3%A1ria_breast_cancer_com_tuning_de_par%C3%A2metros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto 3: Classificação binária brest cancer com tuning dos parâmetros

## Etapa 1: Importação das bibliotecas

In [6]:
!pip install skorch


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
import numpy as np
import sklearn
import skorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
from skorch import NeuralNetBinaryClassifier

In [8]:
torch.__version__, skorch.__version__, sklearn.__version__

('1.13.1+cpu', '0.13.0', '1.0.2')

## Etapa 2: Base de dados

In [9]:
np.random.seed(123)
torch.manual_seed(123)

In [10]:
previsores = pd.read_csv('./databases/entradas_breast.csv')
classe = pd.read_csv('./databases/saidas_breast.csv')

In [11]:
previsores = np.array(previsores, dtype = 'float32')
classe = np.array(classe, dtype = 'float32').squeeze(1)

In [12]:
previsores.shape

(569, 30)

In [13]:
classe.shape

(569,)

## Etapa 3: Classe para estrutura da rede neural

**\*\* ATUALIZAÇÃO JAN/2022 \*\*** : na versão atual do Skorch, os resultados da rede neural devem ser retornados sem ativação, ou seja, sem a camada sigmoide no final. Com isto, a função de custo deve ser `BCEWithLogitsLoss`.

In [14]:
class classificador_torch(nn.Module):
  def __init__(self, activation, neurons, initializer):
    super().__init__()
    # 30 -> 16 -> 16 -> 1
    self.dense0 = nn.Linear(30, neurons)
    initializer(self.dense0.weight)
    self.activation0 = activation
    self.dense1 = nn.Linear(neurons, neurons)
    initializer(self.dense1.weight)
    self.activation1 = activation
    self.dense2 = nn.Linear(neurons, 1)
    initializer(self.dense2.weight)
    # self.output = nn.Sigmoid() ** ATUALIZAÇÃO (ver detalhes no texto acima) **

  def forward(self, X):
    X = self.dense0(X)
    X = self.activation0(X)
    X = self.dense1(X)
    X = self.activation1(X)
    X = self.dense2(X)
    # X = self.output(X) ** ATUALIZAÇÃO (ver detalhes no texto acima) **
    return X

## Etapa 4: Skorch

In [15]:
classificador_sklearn = NeuralNetBinaryClassifier(module=classificador_torch,
                                                  lr = 0.001,
                                                  optimizer__weight_decay = 0.0001,
                                                  train_split=False)

## Etapa 5: Tuning dos parâmetros

In [16]:
params = {'batch_size': [10],
          'max_epochs': [100],
          'optimizer': [torch.optim.Adam, torch.optim.SGD],
          'criterion': [torch.nn.BCEWithLogitsLoss], #, torch.nn.HingeEmbeddingLoss], # ** ATUALIZAÇÃO **
          'module__activation': [F.relu, F.tanh],
          'module__neurons': [8, 16],
          'module__initializer': [torch.nn.init.uniform]} # _, torch.nn.init.normal_]}

In [17]:
params

{'batch_size': [10],
 'max_epochs': [100],
 'optimizer': [torch.optim.adam.Adam, torch.optim.sgd.SGD],
 'criterion': [torch.nn.modules.loss.BCEWithLogitsLoss],
 'module__activation': [<function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
  <function torch.nn.functional.tanh(input)>],
 'module__neurons': [8, 16],
 'module__initializer': [<function torch.nn.init._make_deprecate.<locals>.deprecated_init(*args, **kwargs)>]}

In [18]:
grid_search = GridSearchCV(estimator=classificador_sklearn, param_grid=params,
                           scoring = 'accuracy', cv = 2)
grid_search = grid_search.fit(previsores, classe)

  epoch    train_loss     dur
-------  ------------  ------
      1    27299.8738  0.0884
      2    24343.2038  0.0677


c:\Users\pc\Documents\GitHub\udemy_deep_learning_pytorch_python\udemy_deep_learning_pytorch_python\.venv\lib\site-packages\ipykernel_launcher.py:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  
c:\Users\pc\Documents\GitHub\udemy_deep_learning_pytorch_python\udemy_deep_learning_pytorch_python\.venv\lib\site-packages\ipykernel_launcher.py:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if __name__ == "__main__":
c:\Users\pc\Documents\GitHub\udemy_deep_learning_pytorch_python\udemy_deep_learning_pytorch_python\.venv\lib\site-packages\ipykernel_launcher.py:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if sys.path[0] == "":


      3    21607.9801  0.0455
      4    19135.1209  0.0457
      5    16931.4981  0.0511
      6    14986.2359  0.0418
      7    13263.7468  0.0407


      8    11731.9746  0.0429
      9    10367.9507  0.0395
     10     9150.4845  0.0452
     11     8060.7275  0.0664
     12     7080.4200  0.0448
     13     6192.2921  0.0422
     14     5382.8836  0.0353
     15     4640.1406  0.0366
     16     3952.5612  0.0357
     17     3309.9056  0.0357
     18     2702.8676  0.0350
     19     2122.0807  0.0341
     20     1558.6194  0.0336
     21     1003.4049  0.0363
     22      478.4398  0.0338
     23      138.4234  0.0333
     24       90.7292  0.0341
     25       88.3919  0.0339
     26       77.7052  0.0302
     27       70.4368  0.0351
     28       64.3089  0.0308
     29       58.6982  0.0351
     30       54.5392  0.0333
     31       52.0839  0.0356
     32       48.6802  0.0351
     33       46.1668  0.0338
     34       44.8264  0.0348
     35       43.1170  0.0366
     36       41.1273  0.0360
     37       40.8537  0.0339
     38       40.7872  0.0341
     39       42.1396  0.0361
     40       39.8864  0.0347
     41   

c:\Users\pc\Documents\GitHub\udemy_deep_learning_pytorch_python\udemy_deep_learning_pytorch_python\.venv\lib\site-packages\ipykernel_launcher.py:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  
c:\Users\pc\Documents\GitHub\udemy_deep_learning_pytorch_python\udemy_deep_learning_pytorch_python\.venv\lib\site-packages\ipykernel_launcher.py:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if __name__ == "__main__":
c:\Users\pc\Documents\GitHub\udemy_deep_learning_pytorch_python\udemy_deep_learning_pytorch_python\.venv\lib\site-packages\ipykernel_launcher.py:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if sys.path[0] == "":


...

      8        2.5785  0.0200
      9        2.5088  0.0286
     10        2.4391  0.0265
     11        2.3696  0.0211
     12        2.3002  0.0221
     13        2.2310  0.0231
     14        2.1619  0.0220
     15        2.0931  0.0220
     16        2.0245  0.0246
     17        1.9562  0.0230
     18        1.8883  0.0236
     19        1.8208  0.0232
     20        1.7538  0.0226
     21        1.6875  0.0230
     22        1.6219  0.0222
     23        1.5571  0.0214
     24        1.4934  0.0227
     25        1.4308  0.0230
     26        1.3697  0.0264
     27        1.3101  0.0237
     28        1.2523  0.0231
     29        1.1966  0.0427
     30        1.1432  0.0291
     31        1.0923  0.0236
     32        1.0442  0.0221
     33        0.9992  0.0248
     34        0.9573  0.0273
     35        0.9187  0.0251
     36        0.8836  0.0236
     37        0.8518  0.0242
     38        0.8235  0.0210
     39        0.7984  0.0231
     40        0.7765  0.0220
     41   

c:\Users\pc\Documents\GitHub\udemy_deep_learning_pytorch_python\udemy_deep_learning_pytorch_python\.venv\lib\site-packages\ipykernel_launcher.py:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  
c:\Users\pc\Documents\GitHub\udemy_deep_learning_pytorch_python\udemy_deep_learning_pytorch_python\.venv\lib\site-packages\ipykernel_launcher.py:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if __name__ == "__main__":
c:\Users\pc\Documents\GitHub\udemy_deep_learning_pytorch_python\udemy_deep_learning_pytorch_python\.venv\lib\site-packages\ipykernel_launcher.py:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if sys.path[0] == "":
c:\Users\pc\Documents\GitHub\udemy_deep_learning_pytorch_python\udemy_deep_learning_pytorch_python\.venv\lib\site-packages\torch\nn\functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprec

      8        3.6861  0.0221
      9        3.6191  0.0230
     10        3.5520  0.0231
     11        3.4850  0.0236
     12        3.4180  0.0252
     13        3.3511  0.0206
     14        3.2841  0.0229
     15        3.2171  0.0227
     16        3.1501  0.0244
     17        3.0832  0.0201
     18        3.0163  0.0231
     19        2.9494  0.0225
     20        2.8825  0.0222
     21        2.8156  0.0226
     22        2.7488  0.0211
     23        2.6820  0.0223
     24        2.6153  0.0236
     25        2.5486  0.0206
     26        2.4820  0.0237
     27        2.4155  0.0205
     28        2.3491  0.0241
     29        2.2828  0.0217
     30        2.2167  0.0231
     31        2.1507  0.0221
     32        2.0849  0.0216
     33        2.0194  0.0246
     34        1.9541  0.0246
     35        1.8892  0.0220
     36        1.8247  0.0222
     37        1.7607  0.0230
     38        1.6972  0.0217
     39        1.6344  0.0231
     40        1.5724  0.0221
     41   

c:\Users\pc\Documents\GitHub\udemy_deep_learning_pytorch_python\udemy_deep_learning_pytorch_python\.venv\lib\site-packages\ipykernel_launcher.py:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  
c:\Users\pc\Documents\GitHub\udemy_deep_learning_pytorch_python\udemy_deep_learning_pytorch_python\.venv\lib\site-packages\ipykernel_launcher.py:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if __name__ == "__main__":
c:\Users\pc\Documents\GitHub\udemy_deep_learning_pytorch_python\udemy_deep_learning_pytorch_python\.venv\lib\site-packages\ipykernel_launcher.py:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  if sys.path[0] == "":


      4    38467.3815  0.0640
      5    28422.5020  0.0643
      6    20474.0236  0.0633
      7    14073.4598  0.0631
      8     8759.0273  0.0669
      9     4118.9129  0.0609
     10      574.8676  0.0650
     11      151.9106  0.0612
     12      122.1280  0.0659
     13      103.7969  0.0649
     14       87.6232  0.0659
     15       72.9820  0.0644
     16       64.6275  0.0641
     17       64.3240  0.0648
     18       55.6824  0.0674
     19       56.6579  0.0668
     20       54.0908  0.0661
     21       60.3754  0.0650
     22       61.2469  0.0614
     23       57.6553  0.0675
     24       46.2579  0.0632
     25       41.6352  0.0628
     26       38.4564  0.0641
     27       37.3743  0.0645
     28       40.1999  0.0638
     29       48.5139  0.0650
     30       54.3047  0.0662
     31       55.0084  0.0643
     32       46.8410  0.0654
     33       51.1857  0.0627
     34       53.3760  0.0647
     35       49.9222  0.0633
     36       50.4891  0.0659
     37   

In [19]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_

In [20]:
melhores_parametros

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.BCEWithLogitsLoss,
 'max_epochs': 100,
 'module__activation': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
 'module__initializer': <function torch.nn.init._make_deprecate.<locals>.deprecated_init(*args, **kwargs)>,
 'module__neurons': 16,
 'optimizer': torch.optim.adam.Adam}

In [21]:
melhor_precisao

0.8383308623671856